In [1]:
import numpy as np
import pandas as pd

%run BSE.py

BSE: Run completed.


In [1]:
def run_single_mm_experiment(
    sess_prefix,
    run_id,
    mm_type,
    mm_params,
    market_type="synthetic",
    end_time=60*10,
    seed=None
):
   
    import numpy as np
    import pandas as pd

    if seed is not None:
        np.random.seed(seed)

    trader_spec = {
        'buyers': [
            ('ZIC', 10)
        ],
        'sellers': [
            ('ZIC', 10)
        ],
        'market_makers': [
            (mm_type, 1, mm_params)
        ]
    }



    if market_type == "synthetic":

        supply = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed'
            }
        ]

        demand = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed'
            }
        ]

        order_sched = {
            'sup': supply,
            'dem': demand,
            'timemode': 'periodic',
            'interval': 1
        }


    elif market_type == "ibm_offset":

        offset_fn = offset_from_file(
            price_offset_filename="offset-ibm-1m-170831.csv",
            time_col=1,
            price_col=5,
            scale_factor=1,
            end_time=end_time
        )

        supply = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed',
                'offsetfn': offset_fn
            }
        ]
        demand = [
            {
                'from': 0,
                'to': end_time,
                'ranges': [(100, 200)],
                'stepmode': 'fixed',
                'offsetfn': offset_fn
            }
        ]
        order_sched = {
                'sup': supply, 
                'dem': demand,
                'timemode': 'periodic',
                'interval': 1
        }

    else:
        raise ValueError("Unknown market type")

    # ----- run market session -----
    session_id = f"{sess_prefix}_{run_id}"

    dump_flags = {
        'dump_blotters': False,
        'dump_lobs': False,
        'dump_tape': False,
        'dump_strats': False,
        'dump_order_books': False,
        'dump_trader_stats': True,
        'dump_avgbals':True
    }

    market_session(
        sess_id=session_id,
        starttime=0,
        endtime=end_time,
        trader_spec=trader_spec,
        order_schedule=order_sched,
        dumpfile_flags=dump_flags,
        sess_vrbs=False
    )


    statsfile = f"{session_id}_avg_balance.csv"
    df = pd.read_csv(statsfile)
    if df.shape[0] == 0:
        profit = 0.0
    else:
        initial_balance = df.iloc[0, 1]
        final_balance = df.iloc[-1, 1]
        profit = final_balance - initial_balance

    return profit


In [2]:
p1 = run_single_mm_experiment(
    sess_prefix="q4_sanity",
    run_id=0,
    mm_type="MMM01",
    mm_params=MMM01_star_params,
    market_type="synthetic",
    end_time=60*10
)

p2 = run_single_mm_experiment(
    sess_prefix="q4_sanity",
    run_id=1,
    mm_type="MMM02",
    mm_params=MMM02_params,
    market_type="synthetic",
    end_time=60*10
)

print("MMM01* profit:", p1)
print("MMM02  profit:", p2)

NameError: name 'MMM01_star_params' is not defined